## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [3]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [4]:
import json
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re
import pytz
from dateutil import parser

## Download

In [5]:
POWER_BI_QUERY_URL = "https://wabi-us-gov-virginia-api.analysis.usgovcloudapi.net/public/reports/querydata"
REQUEST_PARAMS = (("synchronous", "true"), ("Connection", "keep-alive"))
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"a","Entity":"UDF_Disease","Type":0},{"Name":"m","Entity":"Measures 2","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"a"}},"Property":"CityZip"},"Name":"AggregateCases.Locations.City/Zip"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"M2 CaseCount"},"Name":"Measures 2.M2 CaseCount"},{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"M2 14 DayCaseCount"},"Name":"Measures 2.M2 14 DayCaseCount"},{"Arithmetic":{"Left":{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"M2 14 DayCaseCount"}},"Right":{"ScopedEval":{"Expression":{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"M2 14 DayCaseCount"}},"Scope":[]}},"Operator":3},"Name":"Divide(Measures 2.M2 14 DayCaseCount, ScopedEval(Measures 2.M2 14 DayCaseCount, []))"}],"OrderBy":[{"Direction":2,"Expression":{"Measure":{"Expression":{"SourceRef":{"Source":"m"}},"Property":"M2 CaseCount"}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2,3],"Subtotal":1}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Aggregates":[{"Select":2,"Aggregations":[{"Min":{}},{"Max":{}}]},{"Select":1,"Aggregations":[{"Min":{}},{"Max":{}}]}],"Version":1},"ExecutionMetricsKind":1}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"a\\",\\"Entity\\":\\"UDF_Disease\\",\\"Type\\":0},{\\"Name\\":\\"m\\",\\"Entity\\":\\"Measures 2\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"a\\"}},\\"Property\\":\\"CityZip\\"},\\"Name\\":\\"AggregateCases.Locations.City/Zip\\"},{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"m\\"}},\\"Property\\":\\"M2 CaseCount\\"},\\"Name\\":\\"Measures 2.M2 CaseCount\\"},{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"m\\"}},\\"Property\\":\\"M2 14 DayCaseCount\\"},\\"Name\\":\\"Measures 2.M2 14 DayCaseCount\\"},{\\"Arithmetic\\":{\\"Left\\":{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"m\\"}},\\"Property\\":\\"M2 14 DayCaseCount\\"}},\\"Right\\":{\\"ScopedEval\\":{\\"Expression\\":{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"m\\"}},\\"Property\\":\\"M2 14 DayCaseCount\\"}},\\"Scope\\":[]}},\\"Operator\\":3},\\"Name\\":\\"Divide(Measures 2.M2 14 DayCaseCount, ScopedEval(Measures 2.M2 14 DayCaseCount, []))\\"}],\\"OrderBy\\":[{\\"Direction\\":2,\\"Expression\\":{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"m\\"}},\\"Property\\":\\"M2 CaseCount\\"}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0,1,2,3],\\"Subtotal\\":1}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Window\\":{\\"Count\\":500}}},\\"Aggregates\\":[{\\"Select\\":2,\\"Aggregations\\":[{\\"Min\\":{}},{\\"Max\\":{}}]},{\\"Select\\":1,\\"Aggregations\\":[{\\"Min\\":{}},{\\"Max\\":{}}]}],\\"Version\\":1},\\"ExecutionMetricsKind\\":1}}]}","QueryId":"","ApplicationContext":{"DatasetId":"ef76455e-ab17-44dc-95f8-41d0387bae62","Sources":[{"ReportId":"dcf899e4-aaeb-499e-98a2-05a80a572376","VisualId":"4507e8aa9bacd78d1ff8"}]}}],"cancelQueries":[],"modelId":370311}'

In [6]:
headers = {
    "Host": "wabi-us-gov-virginia-api.analysis.usgovcloudapi.net",
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https",
    "ActivityId": "da2bbb53-fe55-43b4-b03a-f3c8bf7bd161",
    "RequestId": "2cab505d-85bd-07c2-1300-5f6c97d4c3c6",
    "X-PowerBI-ResourceKey": "3fa9fb1c-7ed4-4559-a0cf-d0662d779175",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://app.powerbigov.us",
    "DNT": "1",
    "Connection": "keep-alive",
}
json_data = json.loads(REQUEST_BODY)

In [7]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/Users/sgreene/.local/share/virtualenvs/california-coronavirus-scrapers-o-Rl0Txv/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-virginia-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:
data = response.json()

## Parse

In [8]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [9]:
rows = [d for d in ds["PH"][1]["DM1"] if len(d["C"]) > 1]

In [10]:
dict_list = []
for row in rows:
    d = dict(area=row["C"][0], cases=row["C"][1])
    dict_list.append(d)

In [11]:
df = pd.DataFrame(dict_list)

Get the date

In [12]:
raw_date_string = date = data["results"][0]["result"]["data"]["timestamp"]

In [13]:
raw_date_dt = parser.parse(raw_date_string).astimezone(pytz.timezone("US/Pacific"))

In [14]:
county_date = str(raw_date_dt.date())
county_date

'2021-09-10'

In [15]:
df.insert(0, "county", "Ventura")

In [16]:
df["county_date"] = county_date

Replace the  dashes

In [17]:
df["zip"] = df.area.str[:5]

In [18]:
trimmed_df = df.rename(columns={"cases": "confirmed_cases"}).replace(
    " - ", ": ", regex=True
)

## Vet

In [19]:
try:
    assert not len(trimmed_df) > 26
except AssertionError:
    raise AssertionError("Ventura County's scraper has extra rows")

In [20]:
try:
    assert not len(trimmed_df) < 26
except AssertionError:
    raise AssertionError("Ventura County's scraper is missing rows")

## Export

Set date

In [21]:
tz = pytz.timezone("America/Los_Angeles")

In [22]:
today = datetime.now(tz).date()

In [23]:
slug = "ventura"

In [24]:
trimmed_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [25]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [26]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [27]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [28]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)